In [366]:
import tensorflow as tf

In [367]:
import numpy as np
import os 
import time

In [368]:
path_to_file = ('three_artists.txt')

In [369]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(text)))

Length of text: 219954 characters


In [370]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)

97 unique characters
['\n', ' ', '!', '"', '#', '$', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¿', 'É', 'ß', 'á', 'ä', 'é', 'í', 'ï', 'ñ', 'ó', 'ö', 'ú', 'ü', '\u2005', '–', '—', '‘', '’', '…']


In [371]:
#creating and mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [372]:
print('{')
for char, _ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d}, '.format(repr(char), char2idx[char]))
print(' ...\n}')

{
  '\n':   0, 
  ' ' :   1, 
  '!' :   2, 
  '"' :   3, 
  '#' :   4, 
  '$' :   5, 
  "'" :   6, 
  '(' :   7, 
  ')' :   8, 
  '*' :   9, 
  ',' :  10, 
  '-' :  11, 
  '.' :  12, 
  '0' :  13, 
  '1' :  14, 
  '2' :  15, 
  '3' :  16, 
  '4' :  17, 
  '5' :  18, 
  '6' :  19, 
 ...
}


In [373]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'A tornado fle' ---- characters mapped to int ---- > [26  1 71 66 69 65 52 55 66  1 57 63 56]


In [374]:
# The maximum length sentence we want for a single input in characters
seq_length = 150
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

A
 
t
o
r


In [375]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"A tornado flew around my room before you came\nExcuse the mess it made, it usually doesn't rain in\nSouthern California, much like Arizona\nMy eyes don't "
"shed tears, but boy, they pour when\n\nI'm thinkin' 'bout you, ooh no, no, no\nI've been thinkin' bout you, you know, know, know\nI've been thinkin' bout y"
"ou, do you think about me still?\nDo ya, do ya?\n\nOr do you not think so far ahead?\n'Cause I been thinkin' bout forever, ooh\nOr do you not think so far a"
"head?\n'Cause I been thinkin' bout forever, ooh\n\nNo, I don't like you, I just thought you were cool enough to kick it\nGot a beach house I could sell you"
" in Idaho\nSince you think I don't love you, I just thought you were cute\nThat's why I kissed you\nGot a fighter jet, I don't get to fly it though, I'm l"


In [376]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "A tornado flew around my room before you came\nExcuse the mess it made, it usually doesn't rain in\nSouthern California, much like Arizona\nMy eyes don't"
Target data: " tornado flew around my room before you came\nExcuse the mess it made, it usually doesn't rain in\nSouthern California, much like Arizona\nMy eyes don't "


In [377]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 26 ('A')
  expected output: 1 (' ')
Step    1
  input: 1 (' ')
  expected output: 71 ('t')
Step    2
  input: 71 ('t')
  expected output: 66 ('o')
Step    3
  input: 66 ('o')
  expected output: 69 ('r')
Step    4
  input: 69 ('r')
  expected output: 65 ('n')


In [378]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>

In [379]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [380]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 150, 97) # (batch_size, sequence_length, vocab_size)


In [381]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 150, 97)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.576527


In [382]:
model.compile(optimizer='adam', loss=loss)

In [383]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [384]:
EPOCHS=10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
22/22 [==============================] - 40s 2s/step - loss: 4.0392
Epoch 2/10
22/22 [==============================] - 41s 2s/step - loss: 3.0583
Epoch 3/10
22/22 [==============================] - 41s 2s/step - loss: 2.7135
Epoch 4/10
22/22 [==============================] - 40s 2s/step - loss: 2.4718
Epoch 5/10
22/22 [==============================] - 39s 2s/step - loss: 2.3581
Epoch 6/10
22/22 [==============================] - 39s 2s/step - loss: 2.2892
Epoch 7/10
22/22 [==============================] - 39s 2s/step - loss: 2.2245
Epoch 8/10
22/22 [==============================] - 39s 2s/step - loss: 2.1595
Epoch 9/10
22/22 [==============================] - 39s 2s/step - loss: 2.0895
Epoch 10/10
22/22 [==============================] - 39s 2s/step - loss: 2.0246


In [328]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [385]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (64, None, 256)           24832     
_________________________________________________________________
gru_24 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_24 (Dense)             (64, None, 97)            99425     
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [386]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 280

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.25

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

print(generate_text(model, start_string="\n"))

ValueError: Tensor's shape (1, 64, 1024) is not compatible with supplied shape [1, 1, 1024]

In [387]:
generate_text(model, start_string="\n").splitlines()

ValueError: Tensor's shape (1, 64, 1024) is not compatible with supplied shape [1, 1, 1024]